In [1]:
import pandas
import requests


count = 0
length = 50
total_length = 500

bucket_items = {}
group_items = {}
items = pandas.DataFrame()

url = 'https://www.cupoy.com/MixNewsMongoAction.do'
payload = {'op': 'getTopMixNews', 'nationid': 'TW', 'startNewsID': 0, 'len': length}

In [2]:
for idx in range(0, total_length, length):
    payload['startNewsID'] = idx
    payload['len'] = length

    response = requests.get(url, params=payload).json()
    for item in response['items']:
        item_df = pandas.DataFrame({
            'title': item.get('title'),
            'description': item.get('description'),
            'linkurl': item['linkurl'],
            'keywordtrend': item['metrics']['keywordtrend'],
            'uidactscore': item['metrics']['uidactscore'],
            'socialactscore': item['metrics']['socialactscore']
        }, index=[item['itemuid']])

        print(count, item['title'], sep='\t')
        items = items.append(item_df)
    
        for bucketid in item.get('bucketids', []):
            if bucketid not in bucket_items:
                bucket_items[bucketid] = item_df
            else:
                bucket_items[bucketid]= bucket_items[bucketid].append(item_df)

        for groupid in item.get('bucketgrpids', []):
            if groupid not in group_items:
                group_items[groupid] = item_df
            else:
                group_items[groupid]= group_items[groupid].append(item_df)

        count += 1


| 經理人
87	向上溝通的說話哲學：5 分鐘能講完，就別浪費老闆 10 分鐘！ | 經理人
88	下個 10 年，你的公司可能不在了！給領導者的 2020 「求生」建議 | 經理人
89	日本熱傳「警視庁」教你自製口罩 | Qooah
90	Toyota 完成收購「掃把佬」，成最大股東 | Qooah
91	【眾志口罩】香港眾志 10 萬口罩網店 (12/2) 8:00pm 發售 (附連結) | Qooah
92	北海道食熱杯麵好奢侈？放一陣就硬哂軚 | Qooah
93	【卓悅口罩】網店快閃發售：2,000 盒藍鷹牌兒童口罩 (產品直達 Link) | Qooah
94	媽媽自製吞拿魚紅渣頭，網民：X，還有紅魔蟹咖喱飯 | Qooah
95	Galaxy Z Flip 發佈，看看價錢還是算了…… | Qooah
96	Galaxy S20 系列發佈，100倍 Zoom 的魔鬼 | Qooah
97	猛獸咆哮，SSC Tuatara 1775匹超跑鏟地而起 | Qooah
98	【卓悅口罩】免市民排隊之苦，卓悅公佈以手機 App 取籌買口罩！ | Qooah
99	12 家廠商退出 MWC 2020，大會本週五決定應否腰斬！ | Qooah
100	小米10 未發佈先拆解：電池容量比 小米9 多出 1,100mAh！ | Qooah
101	上船發問卷的檢疫官有戴口罩，卻還是確診了.....「鑽石公主號」羣聚感染嚴重，今晨再添39例-風傳媒
102	從別人的故事中學習成長 | 知 識 家
103	雄中70年前差點被血洗！學生挺身跟軍隊火拼，卻遭國民黨汙衊數十年…揭課本沒提的黑歷史-風傳媒
104	享受新加坡米其林餐廳不必出國，聚餐首選親民星級餐廳「莆田」讓人驚呼哪道才是招牌菜-風傳媒
105	名列臺灣五大家族，採礦致富的基隆顏家為何榮景不再？一段忘年異國戀，道盡豪門的興衰…-風傳媒
106	如果你現在不快樂，你一定不在現在 | 知 識 家
107	當你把自己照顧得很好，就會變得很吸引人！ | 知 識 家
108	口罩之亂與民眾無關？黃士修直指「特權人士」：每天有200萬片消失黑洞-風傳媒
109	為何韓國拍得出《寄生上流》臺灣卻不能？臺劇粉專揭「國產好作品」胎死腹中的致命原因-風傳媒
110	韓國導演奉俊昊不為人知的一面：生在流血鎮壓年代，他曾剃頭示威、被政府列入

In [3]:
items.to_csv('topnews.csv')
items

,title,description,linkurl,keywordtrend,uidactscore,socialactscore
68747470733A2F2F746563686E6577732E74772F323032302F30322F31322F636F6D70616C2D656E67696E6565722D72756D6F7265642D746F2D62652D696E6665637465642D776974682D777568616E2D706E65756D6F6E69612D6F6666696369616C2D736169642D69742D7761732D612D72756D6F722F3F75746D5F736F757263653D66625F746E2675746D5F6D656469756D3D66616365626F6F6B,傳仁寶工程師感染武漢肺炎，官方稱這是謠言 | TechNews 科技新報,日前傳出國內 NB 代工大廠仁寶的一位工程師剛從國外返臺，在與英特爾的會議中出現咳嗽症狀，引...,https://technews.tw/2020/02/12/compal-engineer...,0,85,0
68747470733A2F2F746563686E6577732E74772F323032302F30322F31312F6672616E63652D6861732D66696E65642D6170706C652D32376D2D666F722D616464696E672D626174746572792D6D616E6167656D656E742D66656174757265732D746F2D6970686F6E65732F3F75746D5F736F757263653D66625F746E2675746D5F6D656469756D3D66616365626F6F6B,"iPhone「電池門」事件後續，法國對蘋果罰款 2,500 萬歐元 | TechNews 科技新報",您是否記得自 2017 年底爆發的 iPhone「電池門」事件？法國的消費者監管機構自事件爆...,https://technews.tw/2020/02/11/france-has-fine...,0,85,0
68747470733A2F2F746563686E6577732E74772F323032302F30322F31322F7468652D696D706163742D6F662D6368696E61732D65706964656D69632D69732D71756974652D636F6D706C6963617465642D616E642D7468652D726973652D6F662D7468652D70616E656C2D69732D6E6F742D73696D706C652F3F75746D5F736F757263653D66625F746E2675746D5F6D656469756D3D66616365626F6F6B,中國疫情衝擊相當複雜，集邦科技：面板漲勢並不單純 | TechNews 科技新報,由於武漢肺炎疫情衝擊，目前市場對面板將漲價已有共識，但事實上市場價格變化及供需並沒有這麼簡單...,https://technews.tw/2020/02/12/the-impact-of-c...,0,85,0
68747470733A2F2F746563686E6577732E74772F323032302F30322F31322F77686F2D323031392D6E636F762D6E616D65642D636F7669642D31392F3F75746D5F736F757263653D66625F746E2675746D5F6D656469756D3D66616365626F6F6B,WHO：武漢肺炎定名 COVID-19，估首批疫苗 18 個月內備妥 | TechNews ...,世界衛生組織祕書長譚德塞 11 日宣佈為避免武漢肺炎污名化特定地理位置，正式將武漢肺炎命名為...,https://technews.tw/2020/02/12/who-2019-ncov-n...,0,85,0
687474703A2F2F65646D2E626E6578742E636F6D2E74772F626E706F77657262692F3F75746D5F736F757263653D66616365626F6F6B5F626E2675746D5F6D656469756D3D706F737470726F6D6F74652675746D5F63616D706169676E3D323030323138706F7765726269,Power BI Desktop 大數據應用分析實戰|數位行銷學院|數位時代,Power BI Desktop是商業數據分析工具，為Excel Power BI 免費升級...,http://edm.bnext.com.tw/bnpowerbi/?utm_source=...,0,85,0
...,...,...,...,...,...,...
68747470733A2F2F62757A7A6F72616E67652E636F6D2F766964616F72616E67652F323032302F30322F31322F706F7765722D7175657374696F6E2F,【問對問題更有威力】《紐約時報》等媒體競邀的人際溝通專家分享：如何用「好問題」建立好關係 |...,《VO》導讀： 有時候，一個好的問題，就可以觸動對方心底的熱情與快樂，拉近彼此的距離。兩位人...,https://buzzorange.com/vidaorange/2020/02/12/p...,0,10,0
68747470733A2F2F736F66746E73686172652E636F6D2FE9968BE6BA90E5A0B1E5A0B12D3035382F,Soft & Share 開源報報 058 - Soft & Share 線上課程資訊分享,今天的開源報報出爐了\nhttps://softnshare.com/%e9%96%8b%e...,https://softnshare.com/開源報報-058/,0,10,0
68747470733A2F2F736F66746E73686172652E636F6D2F2565392539362538622565362562612539302565352561302562312565352561302562312D3035382F,Soft & Share 開源報報 058 - Soft & Share 線上課程資訊分享,這是 Soft & Share 為訂閱會員所推出的服務，小編週一到週五會每天整理一篇開源專案...,https://softnshare.com/%e9%96%8b%e6%ba%90%e5%a...,0,10,0
68747470733A2F2F6769726C7374796C652E636F6D2F74772F61727469636C652F3134373736382FE8BAABE58886E8AD89E5AD97E8999FE58C85E590AB2D322D312D342DE79BB4E68EA5E8B2B7E4B880E98081E4B8802DE78F8DE785AEE4B8B9E68385E4BABAE7AF80E8B685E69A96E584AAE683A02DE9809933E5A4A92DE8A693E89C9CE88A8BE59C93E9AEAEE5A5B6EFBC8BE6B3B0E6B3B0E58E9AE5A5B6E8B685E580BCE585A5E6898B,身分證字號包含「2、1、4」直接買一送一！珍煮丹情人節超暖優惠「這3天」覓蜜芋圓鮮奶＋泰泰厚...,默默拿出了身份證❤️（#V編）\n\n#GirlsFoodie女生吃貨 #珍煮丹 #情人節買...,https://girlstyle.com/tw/article/147768/身分證字號包...,0,10,0


In [6]:
print('Bucket')
for bucketid in sorted(bucket_items, key=lambda k: len(bucket_items[k]), reverse=True):
    print(bucketid, len(bucket_items[bucketid]))

print('==========================')
print('Group')
for groupid in sorted(group_items, key=lambda k: len(group_items[k]), reverse=True):
    print(groupid, len(group_items[groupid]))


Bucket
career_tw 31
TravelLife_tw 27
health_tw 26
PopularDelicacy_tw 22
car_tw 18
management_tw 17
finance_tw 15
FinancialTechnology_tw 15
FinancialManagement_tw 14
parenting_tw 14
mobile_tw 12
movie_tw 12
robot_tw 11
international_tw 9
info_security_tw 8
BlockChain_tw 8
marketing_tw 6
e-commerce_tw 5
GoodArticle_tw 5
app_tw 4
uiux_tw 4
startups_tw 4
CrossStrait_tw 4
beautysalon_tw 4
apple_tw 3
3c_tw 3
graphic_design_tw 3
product_design_tw 3
cloud_services_tw 2
notebook_tw 2
GameInformation_tw 2
Baseball_tw 2
gender_tw 2
interior_design_tw 2
IoT_TW 1
drama_tw 1
GoogleZone_tw 1
estate_tw 1
fitness_tw 1
audio_tw 1
ReadNovel_tw 1
NationalPension_tw 1
Basketball_tw 1
Group
life_tw 110
tech_tw 99
business_tw 98
Hedonism_tw 88
Babyhome_tw 71
WhiteCollar_tw 67
3CExpert_tw 52
CityTour_tw 38
InternationalFinance_tw 34
Fitness_tw 27
Hipster_tw 19
GamePlayer_tw 14
design_tw 9
Reading_tw 5
Sports_tw 4
Game_tw 2


In [5]:
print('====================================')
print('keywordtrend', items.loc[items['keywordtrend'].idxmax()].to_dict())
print('uidactscore', items.loc[items['uidactscore'].idxmax()].to_dict())
print('socialactscore', items.loc[items['socialactscore'].idxmax()].to_dict())

keywordtrend {'title': '傳仁寶工程師感染武漢肺炎，官方稱這是謠言 | TechNews 科技新報', 'description': '日前傳出國內 NB 代工大廠仁寶的一位工程師剛從國外返臺，在與英特爾的會議中出現咳嗽症狀，引發與會者恐慌。\r\n\r\n\r\n\r\n這位仁寶林口研發中心的員工因為在與上游廠商英特爾的會議中咳嗽，引發相當大關注，當時這名員工也坦言自己剛從國外返臺，曾在香港轉機，令英特爾與會人員嚇得立刻中止開會，並表示，當天出席...', 'linkurl': 'https://technews.tw/2020/02/12/compal-engineer-rumored-to-be-infected-with-wuhan-pneumonia-official-said-it-was-a-rumor/?utm_source=fb_tn&utm_medium=facebook', 'keywordtrend': 0, 'uidactscore': 85, 'socialactscore': 0}
uidactscore {'title': '該「封城擋病毒」、還是「盡力顧經濟」？路透獨家：習近平指示「遏阻疫病作法太超過，別再傷害中國經濟」-風傳媒', 'description': '中美貿易戰好不容易才簽署第一階段協議，橫空出世的武漢肺炎（COVID-2019、嚴重特殊傳染性肺炎）卻又讓中國陷入「封城」、「封省」的疫病恐懼，對於身為「世界工廠」的中國製造業更是影響深遠。面對「防治疫病」與「避免衝擊經濟」的兩難，被視為「中國近年最獨裁領導人」的習近平，似乎已經', 'linkurl': 'https://www.storm.mg/article/2283762', 'keywordtrend': 0, 'uidactscore': 105, 'socialactscore': 0}
socialactscore {'title': '21 世紀了，印度還有種姓制度嗎？用我認識的五位印度青年真實人生告訴你 | BuzzOrange', 'description': '【為什麼挑選這篇文章】 許多人一定會好奇：都已經 21 世紀了，印度還會受到傳統的種姓制度影響嗎？這個問題說來有些複雜，不如就讓我們實際看看真實例子吧！來聽聽這五位印度